In [1]:
import pandas as pd
from prometheus_pandas import query

In [4]:
df = pd.read_csv('Data/prom.csv')
df

,start,levelname,message,finish,delta,source,target,type
0,2023-01-28 00:11:52,INFO,0 PushEvent created,2023-01-28 00:11:53,1.0,u: fortiz,r: fortiz/sat,PushEvent
1,2023-01-28 00:11:53,INFO,1 PushEvent created,2023-01-28 00:11:54,1.0,u: CarlosAlbertoFigueiredo,r: CarlosAlbertoFigueiredo/ServidorGit,PushEvent
2,2023-01-28 00:11:54,INFO,2 PushEvent created,2023-01-28 00:11:55,1.0,u: BernardTatin,r: BernardTatin/ideal-cobol,PushEvent
3,2023-01-28 00:11:55,INFO,3 ForkEvent created,2023-01-28 00:11:57,2.0,u: wuayaba,r: fortiz/sat,ForkEvent
4,2023-01-28 00:11:57,INFO,4 WatchEvent created,2023-01-28 00:12:23,26.0,u: toomastahves,r: IonicaBizau/node.cobol,WatchEvent
...,...,...,...,...,...,...,...,...
533522,2023-01-29 09:01:06,INFO,533538 FollowEvent created,2023-01-29 09:01:06,0.0,u: karthikvee,u: jvermontcapone,FollowEvent
533523,2023-01-29 09:01:06,INFO,533539 FollowEvent created,2023-01-29 09:01:06,0.0,u: karthikvee,u: jimkaprod,FollowEvent
533524,2023-01-29 09:01:06,INFO,533540 FollowEvent created,2023-01-29 09:01:07,1.0,u: karthikvee,u: FraBle,FollowEvent
533525,2023-01-29 09:01:07,INFO,533541 FollowEvent created,2023-01-29 09:01:07,0.0,u: karthikvee,u: Sn3akyP3t3,FollowEvent


In [5]:
p = query.Prometheus('http://localhost:9090')

In [62]:
def select_query(p, metric, init, endt, step):
    
    it = pd.Timestamp(init).strftime('%Y-%m-%dT%H:%M:%SZ')
    et = pd.Timestamp(endt).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    if metric == 'cpu':
        return p.query_range('max(max_over_time(gitlab_sli:gitlab_service_saturation:ratio{component="cpu"}[1m]))',
                  it, 
                  et, 
                  step).mean().iloc[0]
    if metric == 'latency':
        return p.query_range('sum by (instance) (rate(nginx_vts_upstream_request_seconds_total{instance="localhost:8060"}[1m])) \
                  /sum by (instance) (rate(nginx_vts_upstream_requests_total{instance="localhost:8060"}[1m]))',
                  it,
                  et, 
                  step).mean().iloc[0]
        
    if metric == 'memory':
        return p.query_range('clamp_min(clamp_max(1 -((node_memory_MemFree_bytes{instance="localhost:9100"} \
                  + node_memory_Buffers_bytes{instance="localhost:9100"} \
                  + node_memory_Cached_bytes{instance="localhost:9100"}))\
                  /node_memory_MemTotal_bytes{instance="localhost:9100"},1),0)',
                  it,
                  et, 
                  step).mean().iloc[0]
    
    if metric == 'rps':
        return p.query_range('max(avg_over_time(gitlab_sli:code_method_route:workhorse_http_request_count:rate1m[1m]))',
                             it, 
                             et, 
                             step).mean().iloc[0]
    
    

In [45]:
df = df[df['delta'] > 0]
df

,start,levelname,message,finish,delta,source,target,type
0,2023-01-28 00:11:52,INFO,0 PushEvent created,2023-01-28 00:11:53,1.0,u: fortiz,r: fortiz/sat,PushEvent
1,2023-01-28 00:11:53,INFO,1 PushEvent created,2023-01-28 00:11:54,1.0,u: CarlosAlbertoFigueiredo,r: CarlosAlbertoFigueiredo/ServidorGit,PushEvent
2,2023-01-28 00:11:54,INFO,2 PushEvent created,2023-01-28 00:11:55,1.0,u: BernardTatin,r: BernardTatin/ideal-cobol,PushEvent
3,2023-01-28 00:11:55,INFO,3 ForkEvent created,2023-01-28 00:11:57,2.0,u: wuayaba,r: fortiz/sat,ForkEvent
4,2023-01-28 00:11:57,INFO,4 WatchEvent created,2023-01-28 00:12:23,26.0,u: toomastahves,r: IonicaBizau/node.cobol,WatchEvent
...,...,...,...,...,...,...,...,...
533511,2023-01-29 09:01:02,INFO,533527 FollowEvent created,2023-01-29 09:01:03,1.0,u: karthikvee,u: vivekraghupathy,FollowEvent
533515,2023-01-29 09:01:03,INFO,533531 FollowEvent created,2023-01-29 09:01:04,1.0,u: karthikvee,u: mharth,FollowEvent
533519,2023-01-29 09:01:04,INFO,533535 FollowEvent created,2023-01-29 09:01:05,1.0,u: karthikvee,u: tomverin,FollowEvent
533521,2023-01-29 09:01:05,INFO,533537 FollowEvent created,2023-01-29 09:01:06,1.0,u: karthikvee,u: esabarinath,FollowEvent


In [63]:
df.loc[:,'cpu'] = df.apply(lambda x: select_query(p,'cpu', x['start'], x['finish'], x['delta']), axis=1)
df.loc[:,'mem'] = df.apply(lambda x: select_query(p,'memory', x['start'], x['finish'], x['delta']), axis=1)
df.loc[:,'lat'] = df.apply(lambda x: select_query(p,'latency', x['start'], x['finish'], x['delta']), axis=1)
df

/tmp/ipykernel_10367/2631845424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cpu'] = df.apply(lambda x: select_query(p,'cpu', x['start'], x['finish'], x['delta']), axis=1)
/tmp/ipykernel_10367/2631845424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mem'] = df.apply(lambda x: select_query(p,'memory', x['start'], x['finish'], x['delta']), axis=1)
/tmp/ipykernel_10367/2631845424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,start,levelname,message,finish,delta,source,target,type,cpu,mem,lat
0,2023-01-28 00:11:52,INFO,0 PushEvent created,2023-01-28 00:11:53,1.0,u: fortiz,r: fortiz/sat,PushEvent,0.128222,0.637972,0.086000
1,2023-01-28 00:11:53,INFO,1 PushEvent created,2023-01-28 00:11:54,1.0,u: CarlosAlbertoFigueiredo,r: CarlosAlbertoFigueiredo/ServidorGit,PushEvent,0.128222,0.637972,0.086000
2,2023-01-28 00:11:54,INFO,2 PushEvent created,2023-01-28 00:11:55,1.0,u: BernardTatin,r: BernardTatin/ideal-cobol,PushEvent,0.128222,0.654481,0.086000
3,2023-01-28 00:11:55,INFO,3 ForkEvent created,2023-01-28 00:11:57,2.0,u: wuayaba,r: fortiz/sat,ForkEvent,0.128222,0.670991,0.086000
4,2023-01-28 00:11:57,INFO,4 WatchEvent created,2023-01-28 00:12:23,26.0,u: toomastahves,r: IonicaBizau/node.cobol,WatchEvent,0.164167,0.673686,0.076409
...,...,...,...,...,...,...,...,...,...,...,...
533511,2023-01-29 09:01:02,INFO,533527 FollowEvent created,2023-01-29 09:01:03,1.0,u: karthikvee,u: vivekraghupathy,FollowEvent,0.511167,0.770209,0.053095
533515,2023-01-29 09:01:03,INFO,533531 FollowEvent created,2023-01-29 09:01:04,1.0,u: karthikvee,u: mharth,FollowEvent,0.511167,0.770209,0.053095
533519,2023-01-29 09:01:04,INFO,533535 FollowEvent created,2023-01-29 09:01:05,1.0,u: karthikvee,u: tomverin,FollowEvent,0.511167,0.770209,0.053095
533521,2023-01-29 09:01:05,INFO,533537 FollowEvent created,2023-01-29 09:01:06,1.0,u: karthikvee,u: esabarinath,FollowEvent,0.511528,0.770209,0.053095


In [72]:
c = df[['source', 'cpu']].groupby('source').mean()

In [73]:
m = df[['source', 'mem']].groupby('source').mean()

In [74]:
l = df[['source', 'lat']].groupby('source').mean()

In [80]:
ds = c.join(m).join(l)
ds = ds.reset_index()
ds

,source,cpu,mem,lat
0,u: 0532,0.463051,0.755168,0.040190
1,u: 0x00,0.472299,0.752197,0.043218
2,u: 12-tp9d,0.414650,0.739494,0.028374
3,u: 234x,0.442364,0.747136,0.036066
4,u: 4148,0.426614,0.755247,0.035511
...,...,...,...,...
1518,u: zenware,0.508449,0.764717,0.050481
1519,u: zgrep,0.423675,0.711034,0.032695
1520,u: zn,0.440803,0.749883,0.035325
1521,u: zsim0n,0.457283,0.750886,0.040109


In [81]:
ds.to_csv('Data/avg_resource_utilisation.csv', index=False)